In [ ]:
#required imports
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [ ]:
#open data file and read data
with open("ASTR19_S22_group_project_data.txt", encoding="utf-8") as file:
    raw_data = file.read()

#close file when done
file.close()

In [ ]:
#init array to store each line
lineArray = []
#init place for the popped values to go that is not terminal output
junk = ""

#split raw data into lines, store in prepared array
lineArray = str.split(raw_data, "\n")

#remove empty strings from data
for i in lineArray:
    if i == "":
        lineArray.remove(i)

#remove header information in first three lines
junk = lineArray.pop(0)
junk = lineArray.pop(0)
junk = lineArray.pop(0)

In [ ]:
#init arrays to hold components of the data
dayArray = []
timeArray = []
rawDataArray = []
hold = []

#split data components and place in arrays
for i in lineArray:
    hold = str.split(i, " ")
    #this split method is not ideal, but it's been ages and all the stuff I remember is in Java.
    #as such, this loop checks for blank strings and removes them.
    for j in hold:
        if j == '':
            hold.remove('')
    #still in the for loop, add each separated data component to the discrete arrays 
    dayArray.append(hold[0])
    timeArray.append(hold[1])
    rawDataArray.append(hold[2])

In [ ]:
#initialize array to hold time data, and one to hold the data that has been cast
decimalDayArray = []
dataArray = []
#init a holder and counter
temp = 0

#store time as float, with minutes and hours represented as decimal portions of days
#cast each int day to float
for i in dayArray:
    decimalDayArray.append(float(i))

#reset for use as counter
temp = 0

for i in timeArray:
    hold = str.split(i, ":")
    decimalTemp = (float(hold[0]) / 24) + ((float(hold[1]) / 24) / 60)
    decimalDayArray[temp] = decimalDayArray[temp] + decimalTemp
    temp = temp + 1

#cast rawDataArray strings to float into new array
for i in rawDataArray:
    dataArray.append(float(i))

In [ ]:
#plot the data by itself w/ error
#from assignment: "assuming the root mean squared experimental error on the height of the tide is 0.25 ft"
f, ax = plt.subplots()
ax.errorbar(decimalDayArray, 
            dataArray, 
            yerr=.25, 
            fmt='o'
           )
ax.set_xlabel('Time(days)')
ax.set_ylabel('Tide Height(ft)')

'''
#number of points
nPoints = len(decimalDayArray)

#set a range of x values
x_approx = np.linspace(0, 45., nPoints)

#fill approximate function
y_approx = []
for x in x_approx:
    y_approx.append(5 * (np.sin(15 * x) * np.sin(45 * x + 15)) + 2.5)

#plot the approximate function
ax.plot(x_approx, np.asarray(y_approx), "r-")
'''

def f_line(x, a, b, c, d, e, f):
    return  a * (np.sin(b * x + c) * np.sin(d * x + e)) + f

params, params_cov = optimize.curve_fit(f_line, decimalDayArray, dataArray, bounds = (-np.inf, np.inf),
                                        p0 = [-5, 7.5, 4, 60, 30, 2.5])
    
ax.plot(decimalDayArray, f_line(np.asarray(decimalDayArray), *params), 
        label=f"a = {params[0]:.2f}, b = {params[1]:.2f}, c = {params[2]:.2f}, d = {params[3]:.2f}, e = {params[4]:.2f}, f = {params[5]:.2f}")
ax.legend()
f.set_facecolor("white")

#f.savefig("test output.png", bbox_inches = "tight", dpi = 300)

In [ ]:
#replot as histogram
f, axHist = plt.subplots()

#get mean
avg = np.mean(dataArray)

#init array to hold diffs
meanDiff = []

#get set of differences from mean
for i in dataArray:
    meanDiff.append(i - avg)

#plot differences from mean
axHist.hist(dataArray, 20)
axHist.set_xlabel("Deviation from average (ft)")
axHist.set_ylabel("Frequency of occurrence")

axHist.plot(meanDiff[28] + 2.0, .5, "ro")
axHist.annotate("Eruption", 
                (meanDiff[28] + 2.0, .5), 
                (meanDiff[28] + 5, 6.5),
                arrowprops = dict(arrowstyle = "fancy")
               )

f.set_facecolor("white")
f.savefig("test hist.png", bbox_inches = "tight", dpi = 300)

In [ ]:
#determine standard deviation of data and significance of the tsunami
#calculate standard deviation
std = np.std(dataArray)

#find ratio of difference from mean to standard deviation
nSigma = (meanDiff[28] + 2.0) / std

#print results
print(f"This tsunami was a {nSigma:.2f} sigma event!")